In [176]:
import pandas as pd
import numpy as np
import plotly.express as px

from scipy.stats import norm
from scipy.optimize import minimize
from IPython.display import clear_output
from functools import partial
from tqdm import tqdm

# Schaefer or Fox?

One of the first management models you'll learn about are biomass dynamics models. The basic idea of these is simple - the amount of surplus production is related (nonlinearly) to the biomass of your stock. Schaefer's model is one of these and it looks like:

$$B_{n+1} = B_{n} + rB_{n}(1-B_{n}/K) - C$$

where $B_{n}$ is the biomass at time $n$, $r$ and $K$ define how the biomass grows, and $C$ is the catch. $r$ acts as a kind of "how fast does the stock grow when it's unlimited" and $K$ acts as a kind of carrying capacity. 

The key idea with the biomass dynamics models is that there is a specific value of $B_{n}$ at which $rB_{n}(1-B_{n}/K)$ is the largest - i.e. there is the largest surplus yield. (Note that if $C=rB_{n}(1-B_{n}/K)$ you acheive equilibrium). This maximum "sustainable" yield - or MSY - becomes the largest amount you can fish consistently without driving the stock into the ground.

To see where it is for Schaefer, let's plot of simple surplus yield curve:

In [8]:
def schaefer_surplus(params, B):
    K, r, *_ = params
    return r * B * (1 - B / K)

K, r = 100000, 0.01
B = np.arange(0, 100000, 1000) + 1000
px.line(
    x=B,
    y=schaefer_surplus([K, r], B),
    title="Schaefer surplus",
    labels={"x": "Biomass", "y": "Surplus"}
)


From here we can see that our MSY is occuring at a biomass of $K/2$ and if you do the math you'll note that the MSY is actually $Kr/4$.

Alright, so far so good. But there's another biomass dynamics model - the Fox model (or at least one variant of it). This one has a different surplus yield term:

$$B_{n+1} = B_{n} + rB_{n}(-\ln{(B_{n}/K)}) - C$$

Let's see what this one looks like:


In [11]:
def fox_surplus(params, B):
    K, r, *_ = params
    return r * B * (- np.log(B / K))

K, r = 100000, 0.01
B = np.arange(0, 100000, 1000) + 1000
px.line(
    x=B,
    y=fox_surplus([K, r], B),
    title="Schaefer surplus",
    labels={"x": "Biomass", "y": "Surplus"}
)

Interesting! We can see from this that Fox predicts a higher MSY at lower biomass for the same parameters as Schaefer. Specifically it is predicting a MSY of $Kr/e$ at a biomass of $K/e$.

Now this leads us to a conundrum. Which model should I use? The obvious answer would be - whichever fits my data - but fisheries data is rarely so kind. Sometimes both seem to fit just about as well and there's obviously a lot more going on that just what $r$ and $K$ can describe. However if I choose Schaefer and Fox would work I basically cheat a bunch of fishermen about of a better livelihood. But if Fox is wrong I could drive their stock into the ground by overfishing. So what should we do?

Well let's ask the question a different way. We know that the way fisheries are managed is that every few years folks perform a stock assessment and set catch levels. So we're not dealing with a once and done kind of situation, instead we're dealing with a *control loop* and that means that there's opportunity for corrective feedback. Perhaps if we choose Fox when the stock is "really" a Scheafer, the Fox control loop will still be alright. 

To find out let's build some simulations. 

## Simulation

We're going to model our fishery as a `Stock` (which comprises the fish) and a `Fishery` (which comprises the managers and fishermen). The interaction between these two is going to be a `catch` designating how much will have been taken from the stock each year. The `Stock` will manage determining how hard it was to get that `catch` and what it did to the underlying biomass of the fishery while the `Fishery` will determine how to set that `catch` and how much error there is in the `catch` in the first place.

Let's start with the `Stock`.

In [45]:
class SchaeferStock(object):
    def __init__(self, K, r, q, p, K_error, r_error, q_error):
        self.K = K
        self.r = r
        self.q = q
        self.p = p
        self.K_error = K_error if K_error > 0 else 1e-10
        self.r_error = r_error if r_error > 0 else 1e-10
        self.q_error = q_error if q_error > 0 else 1e-10

    def reset(self):
        self.physical = {
            'biomass': [self.K * self.p],
            'K': [],
            'r': [],
            'q': []
        }
        self.measureables = {
            'catch': [],
            'effort': []
        }

    def update_stock_stats(self, K, r, q, B, catch, effort):
        self.physical['K'].append(K)
        self.physical['r'].append(r)
        self.physical['q'].append(q)
        self.physical['biomass'].append(B)

        self.measureables['catch'].append(catch)
        self.measureables['effort'].append(effort)

    def draw_K(self):
        return np.exp(norm.rvs(loc=np.log(self.K), scale=self.K_error, size=1))[0]
    
    def draw_r(self):
        return np.exp(norm.rvs(loc=np.log(self.r), scale=self.r_error, size=1))[0]
    
    def draw_q(self):
        return np.exp(norm.rvs(loc=np.log(self.q), scale=self.q_error, size=1))[0]

    @staticmethod
    def surplus_yield(params, B):
        return schaefer_surplus(params, B)
    
    def fish(self, catch):
        K, r, q = self.draw_K(), self.draw_r(), self.draw_q()
        B = self.physical['biomass'][-1]

        catch = min(B*0.9, catch)
        effort = catch/(q*B)
        B = B + self.surplus_yield((K, r), B) - catch

        self.update_stock_stats(K, r, q, B, catch, effort)

class FoxStock(SchaeferStock):
    @staticmethod
    def surplus_yield(params, B):
        return fox_surplus(params, B)
    
    

Note that we are incorporating error into our fisheries models. This is because, as we said before, $r$ and $K$ don't describe everything going on here. Sometimes recruitment will be great, sometimes it will be poor. Some years the stock will be able to hold more fish, other years it'll be rough. So by incorporating some error we can see how these swings affect the stability of our managed fishery control loops. 

Also we've introduced $q$ which is the catchability coefficient. It determines catch per unit effort or CPUE as:

$$CPUE = qB$$


Alright, let's look for some reasonable values for our parameters.

### Setting Parameters for the Stocks

#### $q$ - Catchability

q is purely relative to our effort so the actual value is not super important (and we're going to assume the fishermen put in the effort regardless of how "hard" it is). What does matter is that our variability isn't all over the place. 

In [81]:
stock = SchaeferStock(
    K=100000, r=0.01, q=0.01, p=0.5, K_error=0.1, r_error=0.1, q_error=0.15
)

np.percentile(np.array([stock.draw_q() for _ in range(10000)]), [25, 50, 75])

array([0.00902856, 0.00999675, 0.01103462])

This seems to give us a resonable amount of swing (~10%).

In [82]:
Q = 0.01
Q_ERROR = 0.15

#### $r$ - Growth Rate

For $r$, we're interested in two things - the degree of error and how quickly it takes a stock to recover from a pretty depleted status. To understand the latter let's start by setting `r_error` to zero (no variance) and just pick rates of growth given low initial stock. 

In [83]:
stock = SchaeferStock(
    K=100000, r=0.01, q=Q, p=0.25, K_error=0, r_error=0, q_error=Q_ERROR
)

stock.reset()
time = [0]
for i in range(100):
    time.append(i+1)
    stock.fish(0)

px.line(
    x=time,
    y=stock.physical['biomass'],
    labels={"x": "Years", "y": "Biomass"},
    title="r=0.01 from K/4"
)

This is suggesting that with $r=0.01$ it would take over 100 years just to reach MSY biomass in an unfished condition. So we'll say this is pretty slow growth.

In [84]:
stock = SchaeferStock(
    K=100000, r=0.1, q=Q, p=0.25, K_error=0, r_error=0, q_error=Q_ERROR
)

stock.reset()
time = [0]
for i in range(100):
    time.append(i+1)
    stock.fish(0)

px.line(
    x=time,
    y=stock.physical['biomass'],
    labels={"x": "Years", "y": "Biomass"},
    title="r=0.1 from K/4"
)

At $r=0.1$ we reach $K$ within 60 years. This will be our fast growth condition. 

In [85]:
stock = SchaeferStock(
    K=100000, r=0.05, q=Q, p=0.25, K_error=0, r_error=0, q_error=Q_ERROR
)

stock.reset()
time = [0]
for i in range(100):
    time.append(i+1)
    stock.fish(0)

px.line(
    x=time,
    y=stock.physical['biomass'],
    labels={"x": "Years", "y": "Biomass"},
    title="r=0.05 from K/4"
)

In [86]:
R_LOW = 0.01
R_ERROR_LOW = 0.15
R_MEDIUM = 0.05
R_HIGH = 0.1

In [87]:
stock = SchaeferStock(
    K=100000, r=R_MEDIUM, q=Q, p=0.25, K_error=0, r_error=0.15, q_error=Q_ERROR
)
np.percentile(np.array([stock.draw_r() for _ in range(10000)]), [25, 50, 75])

array([0.04520866, 0.05011093, 0.05535126])

In [88]:
stock = SchaeferStock(
    K=100000, r=R_HIGH, q=Q, p=0.25, K_error=0, r_error=0.15, q_error=Q_ERROR
)
np.percentile(np.array([stock.draw_r() for _ in range(10000)]), [25, 50, 75])

array([0.09058268, 0.1000697 , 0.11081121])

In [89]:
R_ERROR_MEDIUM = 0.15
R_ERROR_HIGH = 0.15

#### $K$ - Carrying Capacity

This one we're just going to leave as is (fishing level should varying linearly with K). So we just need to determine an error term.

In [90]:
stock = SchaeferStock(
    K=100000, r=R_HIGH, q=Q, p=1, K_error=0.15, r_error=R_ERROR_HIGH, q_error=Q_ERROR
)
np.percentile(np.array([stock.draw_K() for _ in range(10000)]), [25, 50, 75])

array([ 90529.00746142, 100189.02095957, 110876.49105391])

In [91]:
K = 100000
K_ERROR = 0.15

In [93]:
schaefer_high = SchaeferStock(
    K=K, r=R_HIGH, q=Q, p=1, K_error=K_ERROR, r_error=R_ERROR_HIGH, q_error=Q_ERROR
)
schaefer_medium = SchaeferStock(
    K=K, r=R_MEDIUM, q=Q, p=1, K_error=K_ERROR, r_error=R_ERROR_MEDIUM, q_error=Q_ERROR
)
schaefer_low = SchaeferStock(
    K=K, r=R_LOW, q=Q, p=1, K_error=K_ERROR, r_error=R_ERROR_LOW, q_error=Q_ERROR
)
fox_high = FoxStock(
    K=K, r=R_HIGH, q=Q, p=1, K_error=K_ERROR, r_error=R_ERROR_HIGH, q_error=Q_ERROR
)
fox_medium = FoxStock(
    K=K, r=R_MEDIUM, q=Q, p=1, K_error=K_ERROR, r_error=R_ERROR_MEDIUM, q_error=Q_ERROR
)
fox_low = FoxStock(
    K=K, r=R_LOW, q=Q, p=1, K_error=K_ERROR, r_error=R_ERROR_LOW, q_error=Q_ERROR
)

### The First Fishery

Let's start with the simplest fishery - one where the catch is random and unmanaged. 

In [205]:
class RandomFishery(object):
    def __init__(self, catch_center, catch_error):
        self.catch_center = catch_center
        self.catch_error = catch_error

    def reset(self):
        pass

    def fish(self):
        return max(norm.rvs(loc=self.catch_center, scale=self.catch_error, size=1)[0], 0)

In [100]:
random_fishery_high = RandomFishery(catch_center=10000, catch_error=1000)
random_fishery_medium = RandomFishery(catch_center=5000, catch_error=1000)
random_fishery_low = RandomFishery(catch_center=1000, catch_error=1000)

In [101]:
random_fishery_medium.reset()
schaefer_high.reset()
time = [0]
for i in range(20):
    time.append(i+1)
    schaefer_high.fish(random_fishery_medium.fish())

px.line(
    x=time,
    y=schaefer_high.physical['biomass'],
    labels={"x": "Years", "y": "Biomass"},
    title="Schaefer high, random medium"
)

In [103]:
random_fishery_medium.reset()
schaefer_medium.reset()
time = [0]
for i in range(20):
    time.append(i+1)
    schaefer_medium.fish(random_fishery_medium.fish())

px.line(
    x=time,
    y=schaefer_medium.physical['biomass'],
    labels={"x": "Years", "y": "Biomass"},
    title="Schaefer medium, random medium"
)

In [105]:
random_fishery_medium.reset()
schaefer_low.reset()
time = [0]
for i in range(20):
    time.append(i+1)
    schaefer_low.fish(random_fishery_medium.fish())

px.line(
    x=time,
    y=schaefer_low.physical['biomass'],
    labels={"x": "Years", "y": "Biomass"},
    title="Schaefer low, random medium"
)

No surprises here. For our high Schaefer stock we have an $MSY = Kr/4 = 100000 \bullet 0.1 / 4 = 2500$ so in all of these cases our stock is experiencing and over time will become depleted. Therefore we need to introduce some management! However to do this, we have one last thing we need to do - we need to be able to run a stock assessment!

## Stock Assessment

In this case the stock assessment is pretty easy - fit a schaefer or fox model to our catch and effort data.

To do this we first need to be able to calculate our biomass given some assumed parameters.

In [131]:
def predict_biomass(params, catch, surplus_func):
    K, r, _, p, *_ = params
    B = [K*p]
    for i in range(len(catch)-1):
        B.append(B[i] + surplus_func(params, B[i]) - catch[i])
    return np.array(B)

Next, given biomass, effort, and our parameters we need to be able to predict our catch:

In [132]:
def predict_catch(params, B, effort):
    _, _, q, *_ = params
    return B*q*effort

And then we need to compute how likely these catches are, given the observed catches.

In [133]:
def get_NLL(surplus_func, params, catch, effort):
    *_, scale = params
    B = predict_biomass(params, catch, surplus_func)
    catch_pred = predict_catch(params, B, effort)
    return -np.sum(norm.logpdf(np.log(catch), loc=np.log(catch_pred), scale=scale))

With these in hand we can create the function we'll use to get our optimized parameters.

In [134]:
def fit_biomass_dynamics(surplus_func, starting_params, catch, effort):
    NLL = partial(get_NLL, surplus_func)
    return minimize(
        NLL, 
        starting_params, 
        args=(catch, effort), 
        method='Nelder-Mead',
        bounds=((1, float('inf')), (0.0, 1), (0, 1), (1, 1), (0, float('inf')))
    ).x

Let's test this out using our simulation.

In [148]:
random_fishery_medium.reset()
schaefer_medium.reset()
time = [0]
for i in range(20):
    time.append(i+1)
    schaefer_medium.fish(random_fishery_medium.fish())

catch = schaefer_medium.measureables['catch']
effort = schaefer_medium.measureables['effort']

px.scatter(
    x=effort,
    y=catch,
    labels={"x": "Effort", "y": "Catch"},
    title="Schaefer medium, random medium"
)

Alright we're going to cheat a little bit and start our optimization at the values that we used to initialize the sim. Note these won't be the "right" parameters because they don't incorporate our error terms. But this will make it easier to get a sane solution out of the optimization. And in theory the actual managers would put the time in to find these nice starting values by trial and error.

In [149]:
params = [
    K, R_MEDIUM, Q, 1, 100
]
fit_params = fit_biomass_dynamics(
    schaefer_surplus, params, catch, effort
)

/usr/local/lib/python3.10/dist-packages/scipy/stats/_distn_infrastructure.py:2024: RuntimeWarning:

divide by zero encountered in divide



In [150]:
B = predict_biomass(fit_params, catch, schaefer_surplus)
catch_pred = predict_catch(fit_params, B, effort)
obs_df = pd.DataFrame({
    'catch': catch,
    'effort': effort,
    'case': ['obs' for _ in range(len(catch))]
})
pred_df = pd.DataFrame({
    'catch': catch_pred,
    'effort': effort,
    'case': ['pred' for _ in range(len(catch))]
})
df = pd.concat([obs_df, pred_df])
px.scatter(
    df,
    x='effort',
    y='catch',
    color='case',
    labels={"x": "Effort", "y": "Catch"},
    title="Schaefer medium, random medium"
)


In [164]:
print('Fit Params:', round(fit_params[0]), round(fit_params[1], 4))
print('Actual Params:', K, R_MEDIUM)
print('Fit MSY:', round(fit_params[0] * fit_params[1] / 4))
print('Actual MSY:', K * R_MEDIUM / 4)

Fit Params: 92708 0.0799
Actual Params: 100000 0.05
Fit MSY: 1851
Actual MSY: 1250.0


## Managed Fisheries

Let's use our stock assessment tool to create a managed fishery.

In [209]:
class SchaeferFishery(object):
    def __init__(self, starting_catch_limit, catch_error, starting_params):
        self.starting_catch_limit = starting_catch_limit
        self.catch_error = catch_error
        self.starting_params = starting_params

    @staticmethod
    def surplus_func(params, B):
        return schaefer_surplus(params, B)
    
    @staticmethod
    def msy_func(params):
        K, r, *_ = params
        return K * r / 4

    @staticmethod
    def scaling_func(params, catch):
        K, *_ = params
        B_msy = K / 2
        B = predict_biomass(params, catch, schaefer_surplus)[-1]
        return min(1, B/B_msy)

    def reset(self):
        self.catch_limit = self.starting_catch_limit

    def fish(self):
        return max(norm.rvs(loc=self.catch_limit, scale=self.catch_error, size=1)[0], 0)

    def manage(self, measureables):
        catch = measureables['catch']
        effort = measureables['effort']
        params = fit_biomass_dynamics(
            self.surplus_func, self.starting_params, catch, effort
        )
        catch_limit = self.msy_func(params) * (self.scaling_func(params, catch) ** 2)
        self.catch_limit = catch_limit


class FoxFishery(SchaeferFishery):
    @staticmethod
    def surplus_func(params, B):
        return fox_surplus(params, B)
    
    @staticmethod
    def msy_func(params):
        K, r, *_ = params
        return K * r / np.exp(1)
    
    @staticmethod
    def scaling_func(params, catch):
        K, *_ = params
        B_msy = K / np.exp(1)
        B = predict_biomass(params, catch, fox_surplus)[-1]
        return min(1, B/B_msy)

schaefer_medium_fishery = SchaeferFishery(
    starting_catch_limit=5000, catch_error=1000, starting_params=[
        K, R_MEDIUM, Q, 1, 100
    ]
)

fox_medium_fishery = FoxFishery(
    starting_catch_limit=5000, catch_error=1000, starting_params=[
        K, R_MEDIUM, Q, 1, 100
    ]
)


In [210]:
def run_sim(stock, fishery, years_random, years_managed, interim_years):
    fishery.reset()
    stock.reset()

    for _ in range(years_random):
        stock.fish(fishery.fish())

    for i in range(years_managed):
        if i % interim_years == 0:
            fishery.manage(stock.measureables)
        stock.fish(fishery.fish())
    
fishery = schaefer_medium_fishery
stock = schaefer_medium

RUNS = 10
years_random = 20
years_managed = 50
interim_years = 5
dfs = []
for i in tqdm(range(RUNS)):
    run_sim(stock, fishery, years_random, years_managed, interim_years)
    dfs.append(
        pd.DataFrame({
            'catch': stock.measureables['catch'],
            'effort': stock.measureables['effort'],
            'run': [i for _ in range(len(stock.measureables['catch']))],
            'years': [i for i in range(len(stock.measureables['catch']))],
            'biomass': stock.physical['biomass'][:-1]
        })
    )

sos_df = pd.concat(dfs)
px.line(
    sos_df,
    x='years',
    y='biomass',
    color='run',
    labels={"x": "Years", "y": "Biomass"},
    title="Schaefer medium, Schaefer medium"
)


  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/scipy/stats/_distn_infrastructure.py:2024: RuntimeWarning:

divide by zero encountered in divide

/tmp/ipykernel_445/3787539786.py:5: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.10/dist-packages/scipy/stats/_distn_infrastructure.py:2024: RuntimeWarning:

invalid value encountered in subtract

100%|██████████| 10/10 [00:11<00:00,  1.20s/it]


In [211]:
df = sos_df.groupby('years').agg({'catch': ('mean', 'std')})['catch'].reset_index()
px.line(
    df, x='years', y='mean', error_y='std', labels={'mean': 'catch'}, title="Schaefer medium, Schaefer medium"
)

In [212]:
fishery = fox_medium_fishery
stock = fox_medium

RUNS = 10
years_random = 20
years_managed = 50
interim_years = 5
dfs = []
for i in tqdm(range(RUNS)):
    run_sim(stock, fishery, years_random, years_managed, interim_years)
    dfs.append(
        pd.DataFrame({
            'catch': stock.measureables['catch'],
            'effort': stock.measureables['effort'],
            'run': [i for _ in range(len(stock.measureables['catch']))],
            'years': [i for i in range(len(stock.measureables['catch']))],
            'biomass': stock.physical['biomass'][:-1]
        })
    )

fof_df = pd.concat(dfs)
px.line(
    fof_df,
    x='years',
    y='biomass',
    color='run',
    labels={"x": "Years", "y": "Biomass"},
    title="Fox medium, Fox medium"
)

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/scipy/stats/_distn_infrastructure.py:2024: RuntimeWarning:

divide by zero encountered in divide

/tmp/ipykernel_445/1434906612.py:3: RuntimeWarning:

invalid value encountered in log

/tmp/ipykernel_445/3787539786.py:5: RuntimeWarning:

invalid value encountered in log

/tmp/ipykernel_445/3787539786.py:5: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.10/dist-packages/scipy/stats/_distn_infrastructure.py:2024: RuntimeWarning:

invalid value encountered in subtract

100%|██████████| 10/10 [00:14<00:00,  1.47s/it]


In [213]:
df = fof_df.groupby('years').agg({'catch': ('mean', 'std')})['catch'].reset_index()
px.line(
    df, x='years', y='mean', error_y='std', labels={'mean': 'catch'}, title="Fox medium, Fox medium"
)

In [214]:
fishery = schaefer_medium_fishery
stock = fox_medium

RUNS = 10
years_random = 20
years_managed = 50
interim_years = 5
dfs = []
for i in tqdm(range(RUNS)):
    run_sim(stock, fishery, years_random, years_managed, interim_years)
    dfs.append(
        pd.DataFrame({
            'catch': stock.measureables['catch'],
            'effort': stock.measureables['effort'],
            'run': [i for _ in range(len(stock.measureables['catch']))],
            'years': [i for i in range(len(stock.measureables['catch']))],
            'biomass': stock.physical['biomass'][:-1]
        })
    )

sof_df = pd.concat(dfs)
px.line(
    sof_df,
    x='years',
    y='biomass',
    color='run',
    labels={"x": "Years", "y": "Biomass"},
    title="Fox medium, Schaefer medium"
)

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/scipy/stats/_distn_infrastructure.py:2024: RuntimeWarning:

divide by zero encountered in divide

/tmp/ipykernel_445/3787539786.py:5: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.10/dist-packages/scipy/stats/_distn_infrastructure.py:2024: RuntimeWarning:

invalid value encountered in subtract

 10%|█         | 1/10 [00:01<00:11,  1.28s/it]/tmp/ipykernel_445/3787539786.py:5: RuntimeWarning:

invalid value encountered in log

100%|██████████| 10/10 [00:13<00:00,  1.37s/it]


In [215]:
df = sof_df.groupby('years').agg({'catch': ('mean', 'std')})['catch'].reset_index()
px.line(
    df, x='years', y='mean', error_y='std', labels={'mean': 'catch'}, title="Schaefer medium, Fox medium"
)

In [216]:
fishery = fox_medium_fishery
stock = schaefer_medium

RUNS = 10
years_random = 20
years_managed = 50
interim_years = 5
dfs = []
for i in tqdm(range(RUNS)):
    run_sim(stock, fishery, years_random, years_managed, interim_years)
    dfs.append(
        pd.DataFrame({
            'catch': stock.measureables['catch'],
            'effort': stock.measureables['effort'],
            'run': [i for _ in range(len(stock.measureables['catch']))],
            'years': [i for i in range(len(stock.measureables['catch']))],
            'biomass': stock.physical['biomass'][:-1]
        })
    )

fos_df = pd.concat(dfs)
px.line(
    fos_df,
    x='years',
    y='biomass',
    color='run',
    labels={"x": "Years", "y": "Biomass"},
    title="Schaefer medium, Fox medium"
)

  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/scipy/stats/_distn_infrastructure.py:2024: RuntimeWarning:

divide by zero encountered in divide

/tmp/ipykernel_445/3787539786.py:5: RuntimeWarning:

divide by zero encountered in log

/usr/local/lib/python3.10/dist-packages/scipy/stats/_distn_infrastructure.py:2024: RuntimeWarning:

invalid value encountered in subtract

 90%|█████████ | 9/10 [00:15<00:01,  1.60s/it]/tmp/ipykernel_445/1434906612.py:3: RuntimeWarning:

invalid value encountered in log

/tmp/ipykernel_445/3787539786.py:5: RuntimeWarning:

invalid value encountered in log

100%|██████████| 10/10 [00:17<00:00,  1.70s/it]


In [217]:
df = fos_df.groupby('years').agg({'catch': ('mean', 'std')})['catch'].reset_index()
px.line(
    df, x='years', y='mean', error_y='std', labels={'mean': 'catch'}, title="Schaefer medium, Fox medium"
)